# Warming Up

> Easy and not so easy exercises

We use fastai's test library. Our tests always preceed the implementation(s). They should silently go through.

In [ ]:
info = !pip show nbdev
if len(info) < 2:
    !pip install nbdev
    
from fastcore.test import test_eq

## Some Very Short Functions

Write a function `powerOf2(n: int) -> bool` which returns true iff  n is a power of 2.
Hint: No log, no exp. A power of 2, say 8, is a one followed by zeros: 0b1000. Subtracting 1 replaces the leading one with zero and the zeros by one: 8 - 1 = 7 = 0b111.

Here is what `powerOf2` is supposed to do.

In [ ]:
def test_powerOf2():
    test_eq(powerOf2(0), True)
    test_eq(powerOf2(1), True)
    test_eq(powerOf2(7), False)
    test_eq(powerOf2(8), True)

In [ ]:
#collapse
def powerOf2(n: int) -> bool:
    """
    :param n: an integer > = 1
    :return: true if n is a power of two
    """
    return not n & (n - 1)

In [ ]:
test_powerOf2()

Write a function `log2(n: int) -> int` which returns the number of binary digits of n minus 1.

Here is what `log2` is supposed to do.

In [ ]:
def test_log2():
    test_eq(log2(1), 0)
    test_eq(log2(2), 1)
    test_eq(log2(7), 2)
    test_eq(log2(8), 3)

In [ ]:
#collapse
def log2(n: int) -> int:
    """
    :param n: an integer >= 0
    :return: (number of binary digits of n) - 1
    So: log2(1) = 0, log2(2) = 1, log2(3) = 1
    """
    if n < 1:
        raise ValueError
    result = -1
    while n > 0:
        n >>= 1
        result += 1
    return result

In [ ]:
test_log2()

Write a function `gcd(a: int, b: int) -> int` which returns the greatest common divisor of a and b.
How do you deal with negative or zero arguments?

In [ ]:
def test_gcd(*implementations):
    for g in implementations:
        test_eq(g(0, 0), 0)
        test_eq(g(7, 0), 7)
        test_eq(g(0, 7), 7)
        test_eq(g(20, 14), 2)

In [ ]:
#collapse
def gcd(a: int, b: int) -> int:
    """
    :param a: integer
    :param b: integer
    :return: greatest common divisor of a and b
    standard solution
    """
    while b != 0:
        a, b = b, a % b
    return a

Write a recursive function `gcd1(a: int, b: int) -> int` which returns the greatest common divisor of a and b. How deep is the call stack?

In [ ]:
#collapse
def gcd1(a: int, b: int) -> int:
    """
    :param a: integer
    :param b: integer
    :return: greatest common divisor of a and b
    recursive solution
    """
    return a if b == 0 else gcd1(b, a % b)

In [ ]:
test_gcd(gcd, gcd1)

We consider half-open intervals such as u = [a, b) given as a tuple `(a, b)`.
Write a function `intersection(u: (int, int), v: (int, int)) -> (int, int)`
which returns the intersection of the intervals `u` and `v`. Hint: This is a one-liner. No if, no else.
Question: How do you manage the empty interval?

In [ ]:
def test_intersection():
    test_eq(intersection((0, 8), (4, 10)), (4, 8))
    test_eq(intersection((0, 8), (8, 10)), (8, 8))    # empty interval
    test_eq(intersection((0, 8), (9, 10)), (9, 8))    # empty interval
    test_eq(intersection((0, 10), (4, 8)), (4, 8))

In [ ]:
#collapse
def intersection(u: (int, int), v: (int, int)) -> (int, int):
    """
    :param u: half open interval (u0, u1)
    :param v: half open interval (v0, v1)
    :return: intersection of u and v
    Convention: an interval u is empty iff u[0] >= u[1]
    """
    return max(u[0], v[0]), min(u[1], v[1])

Our choice for the emtpy interval: An interval is empty if lower bound $\geq$ upper bound. This makes our program so simple.

In [ ]:
test_intersection()

## Faculty and Fibonacci, to Recurse or not to Recurse?

Write a non-recursive function `faculty(n: int) -> int` which returns n!.

In [ ]:
def test_faculty(*implementations):
    for f in implementations:
        test_eq(f(0), 1)
        test_eq(f(1), 1)
        test_eq(f(2), 2)
        test_eq(f(8), 40320)

In [ ]:
#collapse
def faculty(n: int) -> int:
    """
    :param n: integer
    :return:  nth-faculty
    standard solution
    """
    if n < 0:
        raise ValueError
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result


Write a recursive function `faculty1(n: int) -> int` which returns n!. How deep is the call stack?

In [ ]:
#collapse
def faculty1(n: int) -> int:
    """
    :param n: integer
    :return:  nth-faculty
    recursive solution
    """
    if n < 0:
        raise ValueError
    elif n <= 1:
        return 1
    else:
        return n * faculty1(n - 1)

In [ ]:
test_faculty(faculty, faculty1)

The call stack is $O(n)$. This is no problem because $n!$ can only be computed for small $n$.

Write a function `fibo(n: int) -> int` which returns the n-th fibonacci number

In [ ]:
def test_fibo(*implementations):
    for f in implementations:
        test_eq(f(0), 0)
        test_eq(f(1), 1)
        test_eq(f(2), 1)
        test_eq(f(20), 6765)

In [ ]:
#collapse
def fibo(n: int) -> int:
    """
    :param n: integer >= 0
    :return: n-th Fibonacci number
    standard solution
    """
    if n < 0:
        raise ValueError
    elif n == 0:
        return 0
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

Write a recursive function `fibo(n: int) -> int` which returns the n-th fibonacci number. How deep is the call stack?

In [ ]:
#collapse
def fibo1(n: int) -> int:
    """
    :param n: integer >= 0
    :return: n-th Fibonacci number
    """
    # recursive programming, cool but slow
    if n < 0:
        raise ValueError
    elif n <= 1:
        return n
    else:
        return fibo1(n - 2) + fibo1(n - 1)

In [ ]:
test_fibo(fibo, fibo1)

## Palindromes

Write a non-recursive function `reverse(xs: list) -> None` which reverses the list `xs` in place.

In [ ]:
def test_reverse_in_place():
    xs, ys = [], []
    xs.reverse()
    test_eq(xs, ys)

    xs, ys = [1], [1]
    xs.reverse()
    test_eq(xs, ys)

    xs, ys = [1, 2], [2, 1]
    xs.reverse()
    test_eq(xs, ys)

    xs, ys = [1, 2, 3, 4], [4, 3, 2, 1]
    xs.reverse()
    test_eq(xs, ys)

In [ ]:
#collapse
def reverse(xs: list) -> None:
    """
    :param xs: a list
    :return: None
    Side effect: This function reverses the order of xs
    standard solution
    """
    m = len(xs) // 2
    for i in range(m):
        xs[i], xs[-1 - i] = xs[-1 - i], xs[i]

In [ ]:
test_reverse_in_place()

Write a recursive function `reverse1(xs: list) -> list` which returns the list `xs`in reversed order. How deep is the call stack?

In [ ]:
def test_reverse():
    test_eq(reverse1([]), [])
    test_eq(reverse1([1]), [1])
    test_eq(reverse1([1, 2]), [2, 1])
    test_eq(reverse1([1, 2, 3, 4]), [4, 3, 2, 1])

In [ ]:
#collapse
def reverse1(xs: list) -> list:
    """
    :param xs: a list
    :return: a new list containing xs in reversed order
    same as reverse0, recursive solution
    This produces O(n) new lists!
    """
    return list(xs) if len(xs) <= 1 else [xs[-1]] + reverse1(xs[1: -1]) + [xs[0]]

In [ ]:
test_reverse()

Write a function `normstring(s: str) -> str` which removes all non-letter characters and converts the string to lowercase.

In [ ]:
#collapse
def normstring(s: str) -> str:
    """
    :param s: a string
    :return: keeps ascii letters only and converts s to lower case
    """
    import string
    result = ''
    for c in s:
        if c in string.ascii_letters:
            result += str.lower(c)
    return result

Write a non-recursive function `palindrome1(s: str) -> bool` which returns true iff the string s is a palindrome. Hint: use `normstring` but not `reverse`.

In [ ]:
def test_palindrome(*implementations):
    for p in implementations:
        test_eq(p(""), True)
        test_eq(p("x"), True)
        test_eq(p("xx"), True)
        test_eq(p("xy"), False)
        test_eq(p("Reittier"), True)
        test_eq(p("Reliefpfeiler"), True)
        test_eq(p("Risotto, Sir?"), True)
        test_eq(p("Madam, I'm Adam"), True)
        test_eq(p("Liese, tu Gutes, eil!"), True)
        test_eq(p("Grub Nero nie in Orenburg?"), True)
        test_eq(p("O Genie, der Herr ehre dein Ego!"), True)
        test_eq(p("Lewd I did live, & evil did I dwel?"), True)
        test_eq(p("Eine treue Familie bei Lima feuerte nie"), True)
        test_eq(p("Mad Zeus, no live devil, lived evil on Suez dam"), True)
        test_eq(p("Straw? No, too stupid a fad. I put soot on warts."), True)

In [ ]:
#collapse
def palindrome(xs: str) -> bool:
    """
    :param xs: string
    :return: true if xs is a palindrome
    standard solution
    """
    ys = normstring(xs)
    m = len(ys) // 2
    for i in range(m):
        if ys[i] != ys[-1 - i]:
            return False
    return True

Write a non-recursive function `palindrome1(s: str) -> bool` which returns true iff the string s is a palindrome.
Hint: use `normstring` and `reverse`.

In [ ]:
#collapse
def palindrome1(xs: str) -> bool:
    """
    :param xs: string
    :return: true if xs is a palindrome
    using Python's reverse which only works on lists
    """
    ys = list(normstring(xs))
    zs = list(ys)
    zs.reverse()
    return ys == zs

Write a recursive function `palindrome(s: str) -> bool` which returns true iff the string s is a palindrome.
Hint: use `normstring` but not `reverse`.

In [ ]:
#collapse
def palindrome2(xs: str) -> bool:
    """
    :param xs: string
    :return: true if xs is a palindrome
    recursive solution, normstring only called once
    """

    def pal(ys):
        return True if len(ys) <= 1 else ys[0] == ys[-1] and pal(ys[1:-1])

    return pal(normstring(xs))

Here, `pal`, the function which does the work, is internal to `palindrome2`. So, `normstring` is called only once.

In [ ]:
test_palindrome(palindrome, palindrome1, palindrome2)

## Roman Numbers

Write a function `romans() -> List[str]` which returns the list of all roman numbers from 1 to 4999; the element at zero being " ".

In [ ]:
def test_romans(*implementations):
    for r in implementations:
        rs = r()
        test_eq(len(rs), 5000)
        test_eq(rs[0], '')
        test_eq(rs[1], 'I')
        test_eq(rs[9], 'IX')
        test_eq(rs[4999], 'MMMMCMXCIX' )

In [ ]:
#collapse
from typing import List

def romans() -> List[str]:
    """
    :return: romans numbers from 1 to 4999
    """
    digits0 = ['', 'M', 'MM', 'MMM', 'MMMM']
    digits1 = ['', 'C', 'CC', 'CCC', 'CD', 'D', 'DC', 'DCC', 'DCCC', 'CM']
    digits2 = ['', 'X', 'XX', 'XXX', 'XL', 'L', 'LX', 'LXX', 'LXXX', 'XC']
    digits3 = ['', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX']
   
    result = []
    for d0 in digits0:
        for d1 in digits1:
            for d2 in digits2:
                for d3 in digits3:
                    result.append(d0 + d1 + d2 + d3)
    return result

The following implementation is slightly more elegant. It is the starting point of the conjoin pattern (see xxx). The conjoin pattern is helpful if you don't know the number of nested loops and when the loop criteria depend on some state.

In [ ]:
#collapse
from typing import List

def romans1() -> List[str]:
    """
    :return: romans numbers from 1 to 4999
    """
    digits0 = ['', 'M', 'MM', 'MMM', 'MMMM']
    digits1 = ['', 'C', 'CC', 'CCC', 'CD', 'D', 'DC', 'DCC', 'DCCC', 'CM']
    digits2 = ['', 'X', 'XX', 'XXX', 'XL', 'L', 'LX', 'LXX', 'LXXX', 'XC']
    digits3 = ['', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX']
   
    result = []
    number = 4 * [None]
    for number[0] in digits0:
        for number[1] in digits1:
            for number[2] in digits2:
                for number[3] in digits3:
                    result.append(''.join(number))
    return result

In [ ]:
test_romans(romans, romans1)

## Pascal's Triangle

Write a non-recursive function `bico(n: int) -> List[int]` which returns the binomial coefficients of $(a + b)^n$

In [ ]:
def test_bico(*implementations):
    for b in implementations:
        test_eq(b(0), [1])
        test_eq(b(1), [1, 1])
        test_eq(b(2), [1, 2, 1])
        test_eq(b(5), [1, 5, 10, 10, 5, 1])

In [ ]:
#collapse
def bico(n: int) -> List[int]:
    """
    :param n: an integer >= 0
    :return: coefficients of (a + b) ** n
    """
    triangle = [[1]]  # that's all for n = 0
    for k in range(1, n + 1):
        triangle.append([1])  # append a new line starting with 1
        for i in range(k - 1):  # apply the rule for computing the pascal triangle
            triangle[k].append(triangle[k - 1][i] + triangle[k - 1][i + 1])
        triangle[k].append(1)  # append final 1

    return triangle[n]  # return last line

Write a recursive function `bico(n: int) -> List[int]` which returns the binomial coefficients of $(a + b)^n$. How deep is the call stack?

In [ ]:
#collapse
def bico1(n: int) -> List[int]:
    """
    :param n: an integer >= 0
    :return: coefficients of (a + b) ** n
    recursive implementation
    """
    if n == 0:
        return [1]

    else:
        previous_line = bico1(n - 1)
        this_line = [1]  # set first coefficient = 1
        for i in range(n - 1):  # apply the rule for computing the pascal triangle
            this_line.append(previous_line[i] + previous_line[i + 1])
        this_line.append(1)  # append last coefficient = 1

    return this_line

In [ ]:
test_bico(bico, bico1)

## Lessons Learned

todo